In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!ls

**DESCRIPTION**

THIS FILE WILL PROVIDE RUSSIAN TO ENGLISH (GOOGLE) TRANSLATION FOR ITEM CATEGORIES AND SHOP NAMES WITH TOTAL REVENUE(SALES AMT) AND YYYYMM DATA ON sales_train.csv TABLE PROVIDED.
TO EXTRACT CSV for sales_train.csv after conversion, remove # on kernel. NO MACHINE LEARNING NOR SALES PREDICTION INCLUDED.THANKS.


In [ ]:
import pandas as pd
import pandas_profiling as pdp
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
pd.options.display.float_format = '{:.2f}'.format #指数表記を小数点第2位表記に変更

In [ ]:
#DICTIONARY READ TRANSLATE RUSSIAN TO ENGLISH
dic_shop_EN = pd.read_csv('../input/shop-en-regionadded/shop_en2.csv')
dic_shop_EN.head(20)


*** **↑　店舗名の英文表記読み替え 都市名・リージョン追加　Conv Russian Shopname to En、 City name, region column added
**
　　商品カテゴリの英語名読み替え　Change Item Cat from Russian to En↓******

In [ ]:
#DICTIONARY TO EN for item categories.
dic_itemcat_EN = pd.read_csv('../input/shop-en-regionadded/item_categories_en3.csv',usecols=[0,1,2,3,4,5,6])
dic_itemcat_EN.head(20)


**ここからsales trainデータの検証。（データ長、冒頭・末尾、欠損値のチェック、特徴量）
Verifying sales_train data (length, head/tail, NaN, some characteristics)****

In [ ]:
sales_train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
print(len(sales_train))
#ADD SALES AMT
sales_train["Sales AMT"]= sales_train["item_price"]* sales_train["item_cnt_day"]
#英語名の店名を追記 Add En shopname
sales_train = pd.merge(sales_train, dic_shop_EN, on ="shop_id", how="left")
sales_train.head(10)

In [ ]:
sales_train.tail(10)

In [ ]:
sales_train.isnull().sum()

↑　ここまでSales_trainデータ　Sales Data↑
　　↓ここからtestデータ test data↓

In [ ]:
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
print(len(test))
test.head(5)

In [ ]:
test.tail(5)
test.isnull().sum()

↓ここからitem_categories ↓

In [ ]:
item_categories = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
item_categories.to_csv("item_categories.csv")
print(len(item_categories))
item_categories.head(5)

In [ ]:
item_categories.tail(5)

In [ ]:
item = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
#商品カテゴリに英文を追加。ADD ITEM CAT INTO item df
item = pd.merge(item,dic_itemcat_EN, on="item_category_id", how="left")
print(len(item))
item.head(5)


In [ ]:
item.tail(5)

In [ ]:
item.isnull().sum()

↑ここまで Item   
       ここからshop ↓

In [ ]:
#店舗データ抽出用　shop = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
#shop.to_csv("shop.csv")
#print(len(shop))
#shop.head(5)

In [ ]:
#shop.tail(5)
#shop.isnull().sum()

In [ ]:
sales_train = pd.merge(sales_train, item, on ="item_id", how="left")
delete_columns = ['Column1_y','Column1_x']#なぜかshop_idとitem_category_idが重複するので列削除。(leftmergeの使い方が悪い）Delete unnecessary columns due to left-merges.
sales_train.drop(delete_columns, axis=1, inplace=True)
#DateをYYYYMM単位に簡潔化
sales_train["date"]=pd.to_datetime(sales_train["date"],format='%d.%m.%Y')##Formatを追加。(AH指摘)
sales_train["date"]= sales_train["date"].dt.strftime("%Y%m")

print(len(sales_train))#行数は293,549のままのはず...Length is originally 293,549.Check.
sales_train.head(10)

In [ ]:
sales_train.tail(10)

In [ ]:

sales_train.groupby("date").sum()["Sales AMT"]

In [ ]:
sales_train.groupby("CATEGORY",sort=True).sum()["Sales AMT"]

In [ ]:
sales_train.groupby("REGION", sort=True).sum()["Sales AMT"]

In [ ]:
#plt.figure(figsize=(20,10))
#sns.barplot(x='date',y='Sales AMT', data=sales_train, estimator=sum) #これでSUMを取れるようになる。　

##x_v=sales_train['item_category_name_EN']#ここに条件指定すれば、、、
##y_v=sales_train['Sales AMT']
##sns.barplot(x=x_v,y=y_v)
#plt.xticks(rotation=90)

In [ ]:
#Item PrICE GROUP BY DATE AVG# 上の表の怪しさ検証（単価の平均取ってない？）
sales_train.groupby("date").mean()["item_price"] #★Sales AMT の平均★

In [ ]:
#SALES AMT SUM#
print(sales_train['Sales AMT'].sum())#Sales AMT総合計
print('----------')


#これはうまくいかない#sales_train["date"].sum()[["shop_id","Sales_AMT"]]

In [ ]:
#SALES AMT GROUP BY DATE COUNT#
sales_train.groupby("date").count()["Sales AMT"] #★Sales AMTのCOUNT。★

In [ ]:
#SALES AMT GROUP BY DATE SUM#
sales_train.groupby("date").sum()["Sales AMT"] #★Sales AMTのSUM★

In [ ]:
#pdp.ProfileReport(sales_train)
#実行すると、とても重いです。（GPU浪費）SUPER HEAVY WHEN YOU RUN PROFILE REPORT(>GPU Waste)

In [ ]:
###CSV EXPORT CSVファイル出力するときには＃を外してください。Delete"#" when you export CSV. "###

#sales_train.to_csv("sales_train_EN01.csv")#DLすると６００MB 超えなので注意。VERY HEAVY DATA(>600MB)

In [ ]:
#年月別カテゴリ別売上 Monthly Sales AMT by item cat.
#sales_train.groupby(["date","item_category_name_EN"]).sum()["Sales AMT"]

In [ ]:
print("SUCCESS")